In [1]:
import re
import pandas as pd

In [2]:
def preprocess(data , pattern=24):
    pattern = r"\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s"
    if pattern == 12:
        pattern = r"\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s"
    messages = re.split(pattern, data)[1:]
    dates = re.findall(pattern, data)
    df = pd.DataFrame({
        "user_message": messages,
        "message_date": dates,
    })
    df["date"] = pd.to_datetime(df["message_date"], format="%d/%m/%Y, %H:%M - ")
    users = []
    messages = []
    for message in df["user_message"]:
        # split by : and split to user , message
        entry = re.split("([\w\W]+?):\s",message)
        if entry[1:]: # if it is user message (uname : message)
            users.append(entry[1])
            messages.append(entry[2])
        else: # if it is notification chat without user name
            users.append("notification")
            messages.append(entry[0])
    df["user"] = users
    df["message"] = messages
    df.drop(columns=["user_message"] , inplace=True)

    df["only_date"] = df["date"].dt.date
    df["year"] = df['date'].dt.year
    df["month_num"] = df["date"].dt.month
    df["month"] =  df['date'].dt.month_name()
    df["day"]=  df['date'].dt.day
    df["day_name"] = df["date"].dt.day_name()
    df["hour"] = df["date"].dt.hour
    df["minute"] = df["date"].dt.minute

    period = []
    for hour in df['hour']:
        if hour == 23:
            period.append(str(hour) + "-" + str('00'))
        elif hour == 0:
            period.append(str('00') + "-" + str(hour + 1))
        else:
            period.append(str(hour) + "-" + str(hour + 1))

    df['period'] = period

    return df

<>:16: SyntaxWarning: invalid escape sequence '\w'
<>:16: SyntaxWarning: invalid escape sequence '\w'
C:\Users\cherr\AppData\Local\Temp\ipykernel_21288\807181196.py:16: SyntaxWarning: invalid escape sequence '\w'
  entry = re.split("([\w\W]+?):\s",message)


In [3]:
insomnia_group_chat = open("WhatsApp Chat with Insomniacs.txt", "r", encoding="utf-8")

In [4]:
charan_chat  = open("WhatsApp Chat with Charan  Rkv.txt", "r", encoding="utf-8")

In [5]:
data = charan_chat.read()
data

'03/04/2021, 17:02 - Messages and calls are end-to-end encrypted. Only people in this chat can read, listen to, or share them. Learn more.\n03/04/2021, 17:02 - Your security code with Charan  Rkv changed. Tap to learn more.\n03/04/2021, 17:04 - Your security code with Charan  Rkv changed. Tap to learn more.\n13/04/2021, 07:41 - Your security code with Charan  Rkv changed. Tap to learn more.\n13/04/2021, 08:10 - Your security code with Charan  Rkv changed. Tap to learn more.\n13/04/2021, 08:19 - Your security code with Charan  Rkv changed. Tap to learn more.\n14/04/2021, 07:01 - Your security code with Charan  Rkv changed. Tap to learn more.\n14/04/2021, 22:03 - Your security code with Charan  Rkv changed. Tap to learn more.\n05/05/2021, 20:37 - Charan  Rkv: <Media omitted>\n05/05/2021, 20:37 - Charan  Rkv: <Media omitted>\n05/05/2021, 20:38 - Charan  Rkv: Hi raa balaji\n05/05/2021, 20:38 - Balaji: Hi\n05/05/2021, 20:39 - Charan  Rkv: Answer cheppu raa\n05/05/2021, 20:39 - Balaji: Emo r

In [6]:
df = preprocess(data)

In [7]:
from datetime import timedelta
class TimeSlicer:
    def __init__(self , gap_minutes=30) -> None:
        self.gap_minutes = timedelta(minutes=gap_minutes)


    def slice(self, messages: list):
        if not messages:return "Bro nv manchi vaadiki kaadhu"
        slices = []
        current_slice = [messages[0]]
        for i in range(1 , len(messages)):
            prev_time = messages[i-1][2]
            curr_time = messages[i][2]
            time_diff = curr_time - prev_time
            if time_diff <= self.gap_minutes:
                current_slice.append(messages[i])
            else:
                slices.append(current_slice)
                current_slice = [messages[i]]
        if current_slice: slices.append(current_slice)
        return slices
    

In [8]:
data = (list(zip(df["user"].to_list() , df["message"].to_list() , df["date"].to_list() )))
data[0]

('notification',
 'Messages and calls are end-to-end encrypted. Only people in this chat can read, listen to, or share them. Learn more.\n',
 Timestamp('2021-04-03 17:02:00'))

In [9]:
ts = TimeSlicer()

In [11]:
slices = ts.slice(data)

In [16]:
slices[0][0]

('notification',
 'Messages and calls are end-to-end encrypted. Only people in this chat can read, listen to, or share them. Learn more.\n',
 Timestamp('2021-04-03 17:02:00'))

In [17]:
def slice_text(slices):
    slice_texts = []
    for slice in slices:
        texts = [f"{msg[0] , msg[1] , msg[2]}" for msg in slice]
        slice_text = " ".join(texts)
        slice_texts.append(slice_text)
    return slice_texts
print(slice_text(slices))

["('notification', 'Messages and calls are end-to-end encrypted. Only people in this chat can read, listen to, or share them. Learn more.\\n', Timestamp('2021-04-03 17:02:00')) ('notification', 'Your security code with Charan  Rkv changed. Tap to learn more.\\n', Timestamp('2021-04-03 17:02:00')) ('notification', 'Your security code with Charan  Rkv changed. Tap to learn more.\\n', Timestamp('2021-04-03 17:04:00'))", "('notification', 'Your security code with Charan  Rkv changed. Tap to learn more.\\n', Timestamp('2021-04-13 07:41:00')) ('notification', 'Your security code with Charan  Rkv changed. Tap to learn more.\\n', Timestamp('2021-04-13 08:10:00')) ('notification', 'Your security code with Charan  Rkv changed. Tap to learn more.\\n', Timestamp('2021-04-13 08:19:00'))", "('notification', 'Your security code with Charan  Rkv changed. Tap to learn more.\\n', Timestamp('2021-04-14 07:01:00'))", "('notification', 'Your security code with Charan  Rkv changed. Tap to learn more.\\n', T

In [20]:
import chromadb
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="personal_collection")

In [1]:
import chromadb

client = chromadb.Client()
col = client.create_collection("demo", embedding_function=None)

# Add *manual* embeddings (small vectors)
col.add(
    ids=["1", "2"],
    documents=["Apple is a fruit", "Carrot is a vegetable"],
    embeddings=[[1, 0], [0, 1]]
)

# Query with manual embedding
result = col.query(query_embeddings=[[1, 0]], n_results=1)
print(result)


{'ids': [['1']], 'embeddings': None, 'documents': [['Apple is a fruit']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[None]], 'distances': [[0.0]]}


In [7]:
import faiss
import numpy as np
import json


def store_faiss(messages, embeds, index_path="faiss_index.bin", meta_path="faiss_metadata.json"):
    """
    messages : list of (user, message, timestamp)
    embeds   : list of embedding vectors (list[list[float]])
    """

    # Convert embeddings to float32 numpy
    embed_matrix = np.array(embeds).astype("float32")

    # Build FAISS index
    dim = embed_matrix.shape[1]      # embedding dimension
    index = faiss.IndexFlatL2(dim)   # L2 similarity index

    # Add embeddings to FAISS
    index.add(embed_matrix)

    # Save FAISS index
    faiss.write_index(index, index_path)

    # Save metadata (FAISS cannot save metadata inside index)
    metadata = []
    for (user, msg, ts) in messages:
        metadata.append({
            "user": user,
            "message": msg,
            "timestamp": str(ts)
        })

    with open(meta_path, "w") as f:
        json.dump(metadata, f, indent=4)

    print("FAISS index + metadata saved successfully!")
